# Ontologies
An ontology defines the types of vector tools and classifications that can be used within a project's editor. It is composed of "Root Schema Nodes" which define the kind of tool, the name of the tool, all subclasses that belong to the tool, and more. For instance, a root schema node might define a tool as being a segmentation tool, for a class named "dog", and it has a subclass for selecting if the dog has a leash or not. Root Schema Nodes can be shared between ontologies.
* Helpful Links:
    * [Ontology documentation](https://docs.labelbox.com/docs/ontology-json)
    * [Project Setup Using Ontologies](https://github.com/Labelbox/labelbox-python/blob/develop/examples/project_configuration/project_setup.ipynb)

In [ ]:
!pip install labelbox

In [ ]:
from labelbox import Client, OntologyBuilder, Tool, Classification, Option
import json

In [ ]:
# Add your api key
API_KEY = None
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

### Create Ontology From Normalized JSON
* Users can create ontologies from a json definition of the ontology
* See below `OntologyBuilder` section for more details on constructing the normalized ontology

In [ ]:
# This will automatically create new root schema node
ontology_name = "sdk-ontology"
root_schema_node_cat_normalized_json = {
     'tool': 'polygon', 
     'name': 'cat', 
     'color': 'black'
}

ontology_normalized_json = {"tools" : [root_schema_node_cat_normalized_json], "classifications" : []}
ontology = client.create_ontology(name = ontology_name, normalized_json = ontology_normalized_json)
print(ontology)

### Create Ontology From Existing Root Schema Nodes
* It is often useful to support the same features in multiple ontologies. 
* Labelbox supports this workflow by allowing users to create ontologies using existing root schema nodes.

In [ ]:
# First create the root schema node
root_schema_node_cat = client.create_root_schema_node(root_schema_node_cat_normalized_json)
# When we create the ontology it will not re-create the schema node
ontology = client.create_ontology_from_root_schema_nodes(ontology_name, [root_schema_node_cat.uid])

### Create Ontology From a Mix of New and Existing Root Schema Nodes
* If we want to create a new ontology that expands upon a previous ontology it is helpful to be able to share a portion of the features.
* To do this we will create the new schema ids that we want. Then we will create an ontology from the new list of ids.
* Note that for additional customization you can also combine the normalized json and use the create_ontology() method (not covered here).

In [ ]:
# Create new dog schema id 
root_schema_node_dog_normalized_json = {
     'tool': 'polygon', 
     'name': 'dog', 
     'color': 'black', 
     'classifications': [], 
}
root_schema_node_dog = client.create_root_schema_node(root_schema_node_cat_normalized_json)
# The cat is shared between this new ontology and the one we created previously 
# (ie. the cat root schema node will not be re-created)
ontology = client.create_ontology(ontology_name, [root_schema_node_cat.uid, root_schema_node_dog.uid])

### Read
* We can directly query by id for ontologies and root schema nodes
* We also can search for both by name

In [ ]:
#### Fetch by ID
root_schema_node = client.get_root_schema_node(root_schema_node_cat.uid)
ontology = client.get_ontology(ontology.uid)
print(root_schema_node)
print(ontology)

In [ ]:
#### Search by name
root_schema_node = next(client.get_root_schema_nodes("cat"))
ontology = next(client.get_ontologies(ontology_name))
print(root_schema_node)
print(ontology)

### Update and Delete
- At this time, these options are not supported from the SDK.
- Updating an ontology is dangerous and could cause labels to be hidden. 
    - Use caution when doing so

### Ontology Builder
* The ontology builder is a tool for creating and modifying normalized json

In [ ]:
# Create normalized json with a bounding box and segmentation tool
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="dog"),
    Tool(tool=Tool.Type.SEGMENTATION, name="cat"),
])
# Creating an ontology from this is easy
ontology = client.create_ontology("ontology-builder-ontology", ontology_builder.asdict())
print(json.dumps(ontology.normalized, indent = 2))

* Alternative syntax for defining the ontology via the OntologyBuilder

In [ ]:
# Create
ontology_builder = OntologyBuilder()
# Append tools
tool_dog = Tool(tool=Tool.Type.BBOX, name="dog")
tool_cat = Tool(tool=Tool.Type.SEGMENTATION, name="cat")
ontology_builder.add_tool(tool_dog)
ontology_builder.add_tool(tool_cat)
ontology = client.create_ontology("ontology-builder-ontology", ontology_builder.asdict())
print(json.dumps(ontology.normalized, indent = 2))

* Classifications are supported too (Both for top level and as subclassifications)

In [ ]:
 
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.BBOX, name="dog"),
        Tool(tool=Tool.Type.SEGMENTATION,
             name="cat",
             classifications=[
                 Classification(class_type=Classification.Type.TEXT,
                                instructions="name")
             ])
    ],
    classifications=[
        Classification(class_type=Classification.Type.RADIO,
                       instructions="image_quality",
                       options=[Option(value="clear"),
                                Option(value="blurry")])
    ])
print(json.dumps(ontology_builder.asdict(), indent = 2))

* All Tool objects are constructed the same way:

In [ ]:
bbox_tool = Tool(tool=Tool.Type.BBOX, name="dog_box")
poly_tool = Tool(tool=Tool.Type.POLYGON, name="dog_poly")
seg_tool = Tool(tool=Tool.Type.SEGMENTATION, name="dog_seg")
point_tool = Tool(tool=Tool.Type.POINT, name="dog_center")
line_tool = Tool(tool=Tool.Type.LINE, name="dog_orientation")
ner_tool = Tool(tool=Tool.Type.NER, name="dog_reference")

* Classifications are all constructed the same way (except text which doesn't require options)
* Classifications can be global or subclasses to a tool (ie dog bounding box, with a breed classification)

In [ ]:

text_classification = Classification(class_type=Classification.Type.TEXT,
                                     instructions="dog_name")
radio_classification = Classification(class_type=Classification.Type.CHECKLIST,
                                      instructions="dog_breed",
                                      options=[Option("poodle")])
dropdown_classification = Classification(
    class_type=Classification.Type.DROPDOWN,
    instructions="dog_features",
    options=[Option("short"), Option("fluffy")])
checklist_classification = Classification(
    class_type=Classification.Type.CHECKLIST,
    instructions="background",
    options=[Option("at_park"), Option("has_leash")])